### Import necessary libraries, packages, and dependencies 
- Use os to access the operating system
- Use pandas to create DataFrames and perform analysis 
- Use plotly to generate visuals
- Use dotenv to access .env variables 
- Use sqlalchemy to connect to postgres

In [50]:
import os
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv
from sqlalchemy import create_engine

### Connect to database and retrieve necessary data
- Connect to the 'arbi_db' database 
- Retrieve all data from the 'control' table

In [51]:
# Load environment variables from .env file
load_dotenv()

# Access the variables 
psql_username = os.getenv('PSQL_USERNAME')
psql_password = os.getenv('PSQL_PASSWORD')
psql_host = os.getenv('PSQL_HOST')
psql_port = os.getenv('PSQL_PORT')
db_name = os.getenv('DB_NAME')

# Define the database url
db_url = f"postgresql://{psql_username}:{psql_password}@{psql_host}:{psql_port}/{db_name}" 

# Create the engine object
engine = create_engine(db_url)

# Write ethe SQL query
query = 'SELECT * FROM control'

# Read the SQL query into a DataFrame
control_df = pd.read_sql(query, engine)

# Show the DataFrame's head
control_df.head()


,trade_count,current_datetime,currency,volume,buy_exchange,buy_price,total_purchase_amount,sell_exchange,sell_price,total_sale_amount,profit,spread_percentage
0,1,2024-02-27T21:40:02.609852,bitcoin,1.0,Binance,56965.29,56965.29,Bitstamp,56981.0,56981.0,15.71,0.027578
1,2,2024-02-27T21:40:02.609852,bitcoin,1.0,Binance,56965.29,56965.29,Gemini,56982.2,56982.2,16.91,0.029685
2,3,2024-02-27T21:40:02.609852,bitcoin,1.0,Kraken,56950.20,56950.20,Bitstamp,56981.0,56981.0,30.80,0.054082
3,4,2024-02-27T21:40:02.609852,bitcoin,1.0,Kraken,56950.20,56950.20,Gemini,56982.2,56982.2,32.00,0.056189
4,5,2024-02-27T21:40:02.609852,bitcoin,1.0,Poloniex,56953.20,56953.20,Bitstamp,56981.0,56981.0,27.80,0.048812


### Data Analysis
- Profit Distribution across Trades
- Hourly Profit Trend
- Spread Percentage Analysis 
- Exchange Performance 
- Time Series Analysis

-Note: Limited data collection - Program ran every 30 seconds over a 4 hour window

In [52]:
# Convert the current_datetime column to only show the date
control_df['current_datetime'] = pd.to_datetime(control_df['current_datetime'])
control_df['current_datetime'] = control_df['current_datetime'].apply(lambda x: x.time())

# Set the index to the current_datetime column
control_df.set_index('current_datetime', inplace=True)

control_df

,trade_count,currency,volume,buy_exchange,buy_price,total_purchase_amount,sell_exchange,sell_price,total_sale_amount,profit,spread_percentage
current_datetime,,,,,,,,,,,
21:40:02.609852,1,bitcoin,1.0,Binance,56965.29000,56965.29000,Bitstamp,56981.00000,56981.00000,15.7100,0.027578
21:40:02.609852,2,bitcoin,1.0,Binance,56965.29000,56965.29000,Gemini,56982.20000,56982.20000,16.9100,0.029685
21:40:02.609852,3,bitcoin,1.0,Kraken,56950.20000,56950.20000,Bitstamp,56981.00000,56981.00000,30.8000,0.054082
21:40:02.609852,4,bitcoin,1.0,Kraken,56950.20000,56950.20000,Gemini,56982.20000,56982.20000,32.0000,0.056189
21:40:02.609852,5,bitcoin,1.0,Poloniex,56953.20000,56953.20000,Bitstamp,56981.00000,56981.00000,27.8000,0.048812
...,...,...,...,...,...,...,...,...,...,...,...
21:41:50.383999,96,solana,1.0,Poloniex,108.00800,108.00800,Binance,108.10000,108.10000,0.0920,0.085179
21:41:50.383999,97,solana,1.0,Poloniex,108.00800,108.00800,Bitstamp,108.18710,108.18710,0.1791,0.165821
21:41:50.383999,98,solana,1.0,Poloniex,108.00800,108.00800,Gemini,108.16500,108.16500,0.1570,0.145360


##### Profit Distribution across Trades
- Calculate and visualize the distribution of profits across all trades. This provides insights into the consistency and variability of returns. 

In [53]:
profit_dist = px.histogram(
    control_df, 
    x='profit', 
    nbins=20, 
    title='Profit Distribution', 
    labels={'profit': 'Profit', 'count':'Frequency'})

profit_dist.show()

##### 30 Sec Profit Trend
- Examine the 30 second trend of profits to identify patterns and understand the overall performance of the arbitrage over time.

In [54]:
thirty_sec_profit = control_df.groupby('current_datetime')['profit'].sum().reset_index()
thirty_sec_profit_plot = px.line(
    thirty_sec_profit, 
    x='current_datetime',
    y='profit', 
    title='30 Second Profit Trend',
    labels={'current_datetime':'Time', 'profit':'30 Sec Profit'})

thirty_sec_profit_plot.show()

##### Spread Percentage Analysis
- Explore the spread percentage to understand how much profit is generated relative to the spread. Helps to access the efficieny of the arbitrage strategy. 

In [55]:
spread_percentage_plot = px.scatter(
    control_df, 
    x='spread_percentage',
    y='profit',
    title='Spread Percentage vs Profit',
    labels={'spread_percentage':'Spread Percentage', 'profit':'Profit'})

spread_percentage_plot.show()

##### Exchange Performance
- Evaluate the performance of each exchange in terms of the number of trades and profit. This helps in optimizing the selection of exchanges and diversifying risks.

In [56]:
exchange_performance = control_df.groupby('buy_exchange')['profit'].agg(['count', 'sum']).reset_index()
exchange_performance_plot = px.bar(
    exchange_performance,
    x='buy_exchange',
    y=['count', 'sum'],
    barmode='stack',
    title='Exchange Performance',
    labels={'buy_exchange':'Exchange', 'value':'Value'}
)

exchange_performance_plot.show()

##### Cryptocurrency Performance
- Evaluate the performance of each coin in terms of the number of trades and profit. This helps in optimizing the selection of coins and diversifying risks.

In [57]:
currency_performance = control_df.groupby('currency')['profit'].agg(['count', 'sum']).reset_index()
currency_performance_plot = px.bar(
    currency_performance,
    x='currency',
    y=['count', 'sum'],
    barmode='stack',
    title='Currency Performance',
    labels={'currency':'Currency', 'value':'Value'}
)

currency_performance_plot.show()

##### Time Series Analysis
- Analyze the time series data to identify any patterns or trends in the arbitrage opportunities.

In [58]:
time_series = control_df.reset_index()

time_series_plot = px.line(
    time_series,
    x='current_datetime', 
    y='profit', 
    title='Time Series Analysis',
    labels={'current_datetime': 'Time', 'profit':'Profit'}
)
time_series_plot.show()

### Calculate a minimum profit threshold
- Add transaction fees (use 0.1 percent)
- Adjust the spread_percentage to account for fees

In [61]:
# Calculate total transaction cost for each trade
control_df['buy_fee'] = 0.1  
control_df['sell_fee'] = 0.1  
control_df['total_transaction_cost'] = control_df['buy_fee'] + control_df['sell_fee']

# Filter trades based on a minimum profit threshold in percentage
# Create a list of thresholds values at (.1%, .25%, .5%, .75%, 1%)
thresholds = [0.1, 0.25, 0.5, 0.75, 1]

# Create an empty dictionary to store total profits for each threshold
threshold_profits = {}

# Loop through each threshold and calculate profits
for threshold in thresholds:
    # Adjust spread percentage for expected fees
    control_df['adjusted_spread_percentage'] = control_df['spread_percentage'] - control_df['total_transaction_cost']
    
    # Filter trades by threshold
    profitable_trades = control_df[control_df['adjusted_spread_percentage'] > threshold]

    # Calculate profits 
    profitable_trades['profit'] = profitable_trades['sell_price'] - profitable_trades['buy_price'] - profitable_trades['total_transaction_cost']
    total_profits = profitable_trades['profit'].sum()

    # Store results to dictionary 
    threshold_profits[f'{threshold}% Threshold'] = total_profits

In [62]:
# View profits at various thresholds
threshold_profits

{'0.1% Threshold': 0.0,
 '0.25% Threshold': 0.0,
 '0.5% Threshold': 0.0,
 '0.75% Threshold': 0.0,
 '1% Threshold': 0.0}